# Deep Learning
## HW3 - Problem 1


1.   Full Name: Amir Mohammad Yaghoobian zadeh
2.   Student Number: 401300525



In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torchvision import datasets
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import os
import cv2
import tqdm
import torchvision
from torchvision.transforms import transforms, Lambda

e:\project\ann\new\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


#part A

In [2]:
Resnet50 = torchvision.models.resnet50(pretrained=True, progress=True)
Resnet18 = torchvision.models.resnet18(pretrained=True, progress=True)


In [8]:

transform_train = transforms.Compose([transforms.Resize((32,32)),  #resises the image so it can be perfect for our model.
                                      transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
                                      transforms.RandomRotation(10),     #Rotates the image to a specified angel
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
                                      transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalize all the images
                               ])
 
 
transform = transforms.Compose([transforms.Resize((32,32)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='/cifar10/',
                                             train=True, 
                                             transform=transform_train,
                                             target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1)),
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./cifar10/',
                                            train=False, 
#                                             transform=transforms.ToTensor(),
                                            transform = transform,
                                            target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1)),
                                            download=True)

# Data loader
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=128, 
                                           shuffle=True)

test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=128, 
                                          shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
def train_loop(data_loader, model, loss_func, optimizer, device):
  size = len(data_loader.dataset)
  num_batch = len(data_loader)
  train_loss, correct = 0, 0
  model.train()
  
  for batch ,(X,y) in enumerate(data_loader):
    X = X.to(device)
    y = y.to(device)
    
    pred = model(X)
    loss = loss_func(pred,y)
    train_loss += loss.item()
    
    temp1, temp2 = y.argmax(1), pred.argmax(1)   
    correct += (temp1 == temp2).type(torch.float).sum().item()
    
    # Back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  train_loss /= num_batch
  correct /= size

  print(f"train error:\n Accuracy: {(100*correct):>0.1f}%, Average loss: {train_loss:>8f}")
  return train_loss, correct



def test_loop(data_loader, model, loss_func, device):
  size = len(data_loader.dataset)
  num_batch = len(data_loader)

  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
    
        pred = model(X)
        test_loss += loss_func(pred,y).item()
        
        temp1, temp2 = y.argmax(1), pred.argmax(1)
        correct += (temp1 == temp2).type(torch.float).sum().item()

  test_loss /= num_batch
  correct /= size

  print(f"test error:\n Accuracy: {(100*correct):>0.1f}%, Average loss: {test_loss:>8f} \n")
  return test_loss, correct

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)


cuda:0


In [10]:
Resnet50.fc = nn.Linear(2048, 10)
Resnet50.to(device)

best_acc = 0
epochs = 10

for param in Resnet50.parameters():
    param.requires_grad = False
for param in Resnet50.fc.parameters():
    param.requires_grad = True

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Resnet50.parameters())


for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train_loss, train_acc = train_loop(train_dataloader, Resnet50, loss_func, optimizer, device)
  test_loss, test_acc = test_loop(test_dataloader, Resnet50, loss_func, device)


  if test_acc > best_acc:
    best_acc = test_acc
    torch.save(Resnet50.state_dict(), 'best_Resnet50_model_cifar10_finetune_last_layer.pt')


Epoch: 1
train error:
 Accuracy: 44.0%, Average loss: 1.613603
test error:
 Accuracy: 48.5%, Average loss: 1.482904 

Epoch: 2
train error:
 Accuracy: 44.3%, Average loss: 1.611303
test error:
 Accuracy: 48.8%, Average loss: 1.481307 

Epoch: 3
train error:
 Accuracy: 43.8%, Average loss: 1.612951
test error:
 Accuracy: 49.0%, Average loss: 1.478804 

Epoch: 4
train error:
 Accuracy: 43.7%, Average loss: 1.616584
test error:
 Accuracy: 48.8%, Average loss: 1.484746 

Epoch: 5


KeyboardInterrupt: 

In [17]:
Resnet50 = torchvision.models.resnet50(pretrained=True, progress=True)
Resnet50.fc = nn.Linear(2048, 10)
Resnet50.load_state_dict(torch.load('best_Resnet50_model_cifar10_finetune_last_layer.pt'))
Resnet50.to(device)

test_loss, test_acc = test_loop(test_dataloader, Resnet50, loss_func, device)


test error:
 Accuracy: 47.3%, Average loss: 1.535956 



# part B: knowledge distillation

In [22]:
def train_distillation(data_loader, teacher_model, student_model, loss_func, optimizer, device):
  size = len(data_loader.dataset)
  num_batch = len(data_loader)

  train_loss, correct = 0, 0

  for batch ,(X,y) in enumerate(data_loader):
    X = X.to(device)
    y = y.to(device)
    
    score = teacher_model(X)
    pred = student_model(X)
    
    loss = loss_func(score, pred, y)
    train_loss += loss.item()
    
    temp1, temp2 = y.argmax(1), pred.argmax(1)   
    correct += (temp1 == temp2).type(torch.float).sum().item()
    
    # Back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  train_loss /= num_batch
  correct /= size

  print(f"train error:\n Accuracy: {(100*correct):>0.1f}%, Average loss: {train_loss:>8f}")
  return train_loss, correct



def my_distillation_loss(scores, preds, targets, temperature = 6, alpha=0.5):
    softmax_op = nn.Softmax()
    loss_func = nn.CrossEntropyLoss()
    
    soft_score = softmax_op(scores / temperature)
    soft_pred = softmax_op(preds / temperature)
    loss = (1-alpha) * loss_func(softmax_op(preds), targets) + alpha * temperature**2 * loss_func(soft_pred, soft_score)
    return loss



In [9]:
# set batch size=16
Resnet50 = torchvision.models.resnet50(pretrained=True, progress=True)
Resnet50.fc = nn.Linear(2048, 10)
Resnet50.load_state_dict(torch.load('best_Resnet50_model_cifar10_finetune_last_layer.pt'))

Resnet18 = torchvision.models.resnet18(pretrained=True, progress=True)
Resnet18.fc = nn.Linear(512, 10)

Resnet50.to(device)
Resnet18.to(device)

for param in Resnet18.parameters():
    param.requires_grad = True
    
loss_func = my_distillation_loss
loss_func_BCE = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(Resnet18.parameters())

best_acc = 0
epochs = 20
for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train_loss, train_acc = train_distillation(train_dataloader, Resnet50, Resnet18, loss_func, optimizer, device)
  test_loss, test_acc = test_loop(test_dataloader, Resnet18, loss_func_BCE, device)


  if test_acc > best_acc:
    best_acc = test_acc
    torch.save(Resnet18.state_dict(), 'best_Resnet18_model_cifar10_distilled.pt')


Epoch: 1


C:\Users\amir\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\amir\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\amir\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


train error:
 Accuracy: 48.7%, Average loss: 42.272092
test error:
 Accuracy: 58.1%, Average loss: 2.490658 

Epoch: 2
train error:
 Accuracy: 58.0%, Average loss: 42.210319
test error:
 Accuracy: 60.2%, Average loss: 2.459611 

Epoch: 3
train error:
 Accuracy: 62.0%, Average loss: 42.186183
test error:
 Accuracy: 66.7%, Average loss: 1.978363 

Epoch: 4
train error:
 Accuracy: 64.6%, Average loss: 42.169579
test error:
 Accuracy: 70.7%, Average loss: 1.744897 

Epoch: 5
train error:
 Accuracy: 66.4%, Average loss: 42.157840
test error:
 Accuracy: 70.5%, Average loss: 1.686776 

Epoch: 6
train error:
 Accuracy: 68.0%, Average loss: 42.148061
test error:
 Accuracy: 71.6%, Average loss: 1.586180 

Epoch: 7
train error:
 Accuracy: 69.1%, Average loss: 42.143302
test error:
 Accuracy: 73.1%, Average loss: 1.476059 

Epoch: 8
train error:
 Accuracy: 70.2%, Average loss: 42.137502
test error:
 Accuracy: 73.3%, Average loss: 1.491520 

Epoch: 9
train error:
 Accuracy: 71.2%, Average loss: 42.

results after 3 epochs:

alpha=0.2, temp=0.5: acc=35%
alpha=0.2, temp=2: acc=43%
alpha=0.5, temp=2: acc=43%
alpha=0.5, temp=3.5: acc=58%
alpha=0.5, temp=6: acc=58%
alpha=0.8, temp=6: acc=57%


In [12]:
Resnet18 = torchvision.models.resnet18(pretrained=True, progress=True)
Resnet18.fc = nn.Linear(512, 10)
Resnet18.load_state_dict(torch.load('best_Resnet18_model_cifar10_distilled.pt'))
Resnet18.to(device)

test_loss, test_acc = test_loop(test_dataloader, Resnet18, loss_func, device)


test error:
 Accuracy: 79.0%, Average loss: 1.207464 



# Jim: train Resnet18 from scratch

In [24]:
Resnet18 = torchvision.models.resnet18(pretrained=True, progress=True)

Resnet18.fc = nn.Linear(512, 10)
for param in Resnet18.parameters():
    param.requires_grad = True

Resnet18.to('cuda:0')

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Resnet18.parameters())

best_acc = 0
epochs = 20
for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train_loss, train_acc = train_loop(train_dataloader, Resnet18, loss_func, optimizer, device)
  test_loss, test_acc = test_loop(test_dataloader, Resnet18, loss_func, device)


  if test_acc > best_acc:
    best_acc = test_acc
    torch.save(Resnet18.state_dict(), 'best_Resnet18_model_cifar10_scratch.pt')


Epoch: 1
train error:
 Accuracy: 61.6%, Average loss: 1.113464
test error:
 Accuracy: 74.4%, Average loss: 0.769129 

Epoch: 2
train error:
 Accuracy: 71.4%, Average loss: 0.838485
test error:
 Accuracy: 77.0%, Average loss: 0.692982 

Epoch: 3
train error:
 Accuracy: 74.7%, Average loss: 0.738567
test error:
 Accuracy: 77.4%, Average loss: 0.665003 

Epoch: 4
train error:
 Accuracy: 76.7%, Average loss: 0.675862
test error:
 Accuracy: 80.2%, Average loss: 0.609891 

Epoch: 5
train error:
 Accuracy: 77.8%, Average loss: 0.643458
test error:
 Accuracy: 79.8%, Average loss: 0.603748 

Epoch: 6
train error:
 Accuracy: 79.0%, Average loss: 0.605900
test error:
 Accuracy: 81.0%, Average loss: 0.573960 

Epoch: 7
train error:
 Accuracy: 80.3%, Average loss: 0.571784
test error:
 Accuracy: 81.3%, Average loss: 0.562396 

Epoch: 8
train error:
 Accuracy: 80.2%, Average loss: 0.575524
test error:
 Accuracy: 81.0%, Average loss: 0.568751 

Epoch: 9
train error:
 Accuracy: 81.2%, Average loss: 0.

In [25]:
Resnet18 = torchvision.models.resnet18(pretrained=True, progress=True)
Resnet18.fc = nn.Linear(512, 10)
Resnet18.load_state_dict(torch.load('best_Resnet18_model_cifar10_scratch.pt'))
Resnet18.to(device)

test_loss, test_acc = test_loop(test_dataloader, Resnet18, loss_func, device)


test error:
 Accuracy: 82.9%, Average loss: 0.520595 



چون مدل آموزگار به خوبی تیون نشده است (فقط لایه آخر تیون شده) دقت روش distillation هم خوب نشده است.

# part D

In [19]:
Resnet50 = torchvision.models.resnet50(pretrained=True, progress=True)

Resnet50.fc = nn.Linear(2048, 10)
Resnet50.to(device)

best_acc = 0
epochs = 20

for param in Resnet50.parameters():
    param.requires_grad = True

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Resnet50.parameters())


for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train_loss, train_acc = train_loop(train_dataloader, Resnet50, loss_func, optimizer, device)
  test_loss, test_acc = test_loop(test_dataloader, Resnet50, loss_func, device)


  if test_acc > best_acc:
    best_acc = test_acc
    torch.save(Resnet50.state_dict(), 'best_Resnet50_model_cifar10_finetune_all_network.pt')


Epoch: 1
train error:
 Accuracy: 60.1%, Average loss: 1.165939
test error:
 Accuracy: 72.9%, Average loss: 0.803359 

Epoch: 2
train error:
 Accuracy: 66.4%, Average loss: 0.992998
test error:
 Accuracy: 70.2%, Average loss: 0.885016 

Epoch: 3
train error:
 Accuracy: 62.6%, Average loss: 1.077368
test error:
 Accuracy: 67.7%, Average loss: 0.931816 

Epoch: 4
train error:
 Accuracy: 68.6%, Average loss: 0.892583
test error:
 Accuracy: 74.4%, Average loss: 0.735323 

Epoch: 5
train error:
 Accuracy: 73.6%, Average loss: 0.753297
test error:
 Accuracy: 76.8%, Average loss: 0.678741 

Epoch: 6
train error:
 Accuracy: 75.0%, Average loss: 0.718164
test error:
 Accuracy: 78.6%, Average loss: 0.636451 

Epoch: 7
train error:
 Accuracy: 75.9%, Average loss: 0.698502
test error:
 Accuracy: 79.0%, Average loss: 0.636114 

Epoch: 8
train error:
 Accuracy: 78.6%, Average loss: 0.616628
test error:
 Accuracy: 80.8%, Average loss: 0.561990 

Epoch: 9
train error:
 Accuracy: 79.6%, Average loss: 0.

In [20]:
Resnet50 = torchvision.models.resnet50(pretrained=True, progress=True)
Resnet50.fc = nn.Linear(2048, 10)
Resnet50.load_state_dict(torch.load('best_Resnet50_model_cifar10_finetune_all_network.pt'))
Resnet50.to(device)

test_loss, test_acc = test_loop(test_dataloader, Resnet50, loss_func, device)


test error:
 Accuracy: 83.4%, Average loss: 0.512499 



In [23]:
# set batch size=16
Resnet50 = torchvision.models.resnet50(pretrained=True, progress=True)
Resnet50.fc = nn.Linear(2048, 10)
Resnet50.load_state_dict(torch.load('best_Resnet50_model_cifar10_finetune_all_network.pt'))

Resnet18 = torchvision.models.resnet18(pretrained=True, progress=True)
Resnet18.fc = nn.Linear(512, 10)

Resnet50.to(device)
Resnet18.to(device)

for param in Resnet18.parameters():
    param.requires_grad = True
    
loss_func = my_distillation_loss
loss_func_BCE = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(Resnet18.parameters())

best_acc = 0
epochs = 20
for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train_loss, train_acc = train_distillation(train_dataloader, Resnet50, Resnet18, loss_func, optimizer, device)
  test_loss, test_acc = test_loop(test_dataloader, Resnet18, loss_func_BCE, device)


  if test_acc > best_acc:
    best_acc = test_acc
    torch.save(Resnet18.state_dict(), 'best_Resnet18_model_cifar10_distilled_teacher_tuned_fully.pt')


Epoch: 1


C:\Users\amir\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\amir\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\amir\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


train error:
 Accuracy: 58.3%, Average loss: 40.480923
test error:
 Accuracy: 68.9%, Average loss: 5.476113 

Epoch: 2
train error:
 Accuracy: 67.5%, Average loss: 40.063201
test error:
 Accuracy: 72.1%, Average loss: 5.122643 

Epoch: 3
train error:
 Accuracy: 70.2%, Average loss: 39.968451
test error:
 Accuracy: 73.9%, Average loss: 4.512792 

Epoch: 4
train error:
 Accuracy: 72.2%, Average loss: 39.915855
test error:
 Accuracy: 75.6%, Average loss: 3.962366 

Epoch: 5
train error:
 Accuracy: 73.7%, Average loss: 39.858017
test error:
 Accuracy: 78.2%, Average loss: 3.540137 

Epoch: 6
train error:
 Accuracy: 74.7%, Average loss: 39.830589
test error:
 Accuracy: 78.1%, Average loss: 3.501661 

Epoch: 7
train error:
 Accuracy: 76.0%, Average loss: 39.790882
test error:
 Accuracy: 79.0%, Average loss: 3.204506 

Epoch: 8
train error:
 Accuracy: 76.7%, Average loss: 39.787288
test error:
 Accuracy: 78.7%, Average loss: 3.249483 

Epoch: 9
train error:
 Accuracy: 77.9%, Average loss: 39.

در صورت تیون کردن کل شبکه آموزگار دقت بهتری حاصل می شود. دقت روش distillation نیز نسبت به حالت قبل بهتر خواهد بود.